In [3]:
from utils.dataframes import *

meter_usage

,service_point_id,meter_id,interval_end_datetime,meter_channel,kwh,account_number
0,2300822246,L108605388,10/1/2022 12:00:00 AM,10,0.594,30010320353
1,2300822246,L108605388,10/1/2022 12:15:00 AM,10,0.101,30010320353
2,2300822246,L108605388,10/1/2022 12:30:00 AM,10,0.104,30010320353
3,2300822246,L108605388,10/1/2022 12:45:00 AM,10,0.106,30010320353
4,2300822246,L108605388,10/1/2022 1:00:00 AM,10,0.099,30010320353
...,...,...,...,...,...,...
500275,2300588897,L108607371,9/30/2021 7:00:00 PM,10,1.242,35012790198
500276,2300588897,L108607371,9/30/2021 8:00:00 PM,10,1.202,35012790198
500277,2300588897,L108607371,9/30/2021 9:00:00 PM,10,1.186,35012790198
500278,2300588897,L108607371,9/30/2021 10:00:00 PM,10,1.150,35012790198


## To-do
- Remove `electric_brew.db` if it exists
- Use the same relative path idea for the views themselves and clean up the Parquet querying with Glob

In [4]:
from utils.runtime import connect_to_db

electric_brew = connect_to_db()

electric_brew.execute("SELECT * FROM meter_usage").fetchdf()

2023-11-24 07:52:33,484 | ERROR | Error occurred while creating view 'meter_usage': IO Error: No files found that match the pattern "./data/cmp/curated/meter_usage/**/*.parquet"



IOException: IO Error: No files found that match the pattern "./data/cmp/curated/meter_usage/**/*.parquet"